In [1]:
import pandas  as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_json('dataSet_Culture_06102023-POINT.json')

In [3]:
df2 = df.copy()
df2['polygon_x'] = df['polygon'].apply(lambda x: x['x'])
df2['polygon_y'] = df['polygon'].apply(lambda x: x['y'])
df2['month'] = df['Analysis Date'].apply(lambda x: x.split('-')[1])
df2['day'] = df['Analysis Date'].apply(lambda x: x.split('-')[2])
df2['vegetation'] = (df['indextype'] == 'NDVI') & (df['averagevalue'] >= 0.15)
df2 = df2.drop(['polygon', 'soil_id'] , axis = 1)
df2['combined'] = df2['polygon_x'].astype(str) + '_' + df2['polygon_y'].astype(str) + '_' + df2['year contour'].astype(str)
# Assign unique ID based on the grouped column
df2['field_id'] = df2.groupby('combined').ngroup() + 1
# Drop the combined column
df2 = df2.drop(columns=['combined','polygon_x','polygon_y','Analysis Date'])
label_encoder = LabelEncoder()
label_encoder.fit(df2['culture_name'])
df2['culture_name_encoded'] = label_encoder.transform(df2['culture_name'])

In [4]:
len(df2['culture_name_encoded'].unique())

22

In [5]:
len(df2['culture_name'].unique())

22

In [6]:
encoded_df = pd.get_dummies(df2, columns=['indextype', 'district_name', 'soil_name', 'type_culture_name'])


In [7]:
encoded_df

,averagevalue,year contour,elevation_contour,culture_name,month,day,vegetation,field_id,culture_name_encoded,indextype_NDVI,...,soil_name_Комплекс лугово-сероземных и сероземно-луговых солончаковатых (25-30%) и солонцеватых (25-30%),"soil_name_Комплекс сероземно-луговых и луговых (сазовых) солончаковатых (15-30%) и солонцеватых (15-30%), местами вторично засоленных",soil_name_Лугово-сероземные в сочетании с сероземами северными (малокарбонатными) обыкновенными,soil_name_Луговые и лугово-болотные,"soil_name_Сероземно-луговые, местами солончаковатые",soil_name_Сероземы северные (малокарбонатные) обыкновенные,soil_name_Сероземы северные (малокарбонатные) светлые вторично засоленные и заболоченные,"soil_name_Сероземы северные (малокарбонатные) светлые, местами слабо- и среднесолончаковатые",type_culture_name_Озимовая,type_culture_name_Яровая
0,0.217,2020,1544,Картофель,06,04,True,986,2,True,...,False,False,False,False,False,False,False,False,False,True
1,0.215,2020,1544,Картофель,08,03,True,986,2,True,...,False,False,False,False,False,False,False,False,False,True
2,0.150,2020,1544,Картофель,09,02,True,986,2,True,...,False,False,False,False,False,False,False,False,False,True
3,0.052,2021,1544,Картофель,04,10,False,987,2,True,...,False,False,False,False,False,False,False,False,False,True
4,0.056,2021,1544,Картофель,05,05,False,987,2,True,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,0.055,2022,672,Кукуруза,05,08,False,752,4,False,...,False,False,False,False,True,False,False,False,False,True
22978,0.072,2022,672,Кукуруза,06,07,False,752,4,False,...,False,False,False,False,True,False,False,False,False,True
22979,-0.002,2022,672,Кукуруза,07,02,False,752,4,False,...,False,False,False,False,True,False,False,False,False,True
22980,0.006,2022,672,Кукуруза,08,01,False,752,4,False,...,False,False,False,False,True,False,False,False,False,True


# H2O

In [8]:
import h2o
from h2o.automl import H2OAutoML

In [9]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/dan/Desktop/culture/venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp44_om2aa
  JVM stdout: /tmp/tmp44_om2aa/h2o_dan_started_from_python.out
  JVM stderr: /tmp/tmp44_om2aa/h2o_dan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Bishkek
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_dan_sl875a
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.771 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [10]:
encoded_df.to_csv('encoded_data_culture.csv', index=False)

In [11]:
h2odf = h2o.import_file('encoded_data_culture.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
h2odf.describe(chunk_summary=True)

Rows:22982
Cols:36

Chunk compression summary: 
chunk_type    chunk_name                 count    count_percentage    size      size_percentage
------------  -------------------------  -------  ------------------  --------  -----------------
C0L           Constant long              1571     54.5486             122.7 KB  21.2414
CBS           Binary                     582      20.2083             60.4 KB   10.4476
CXI           Sparse Integers            87       3.02083             10.3 KB   1.77463
C1N           1-Byte Integers (w/o NAs)  339      11.7708             117.6 KB  20.3572
C1S           1-Byte Fractions           212      7.36111             76.9 KB   13.3039
C2            2-Byte Integers            9        0.3125              5.6 KB    0.972157
CUD           Unique Reals               1        0.0347222           1.8 KB    0.311996
C8D           64-bit Reals               79       2.74306             182.5 KB  31.591

Frame distribution summary: 
                 size      number_of_rows    number_of_chunks_per_column    number_of_chunks
---------------  --------  ----------------  -----------------------------  ------------------
127.0.0.1:54321  577.8 KB  22982             80                             2880
mean             577.8 KB  22982             80                             2880
min              577.8 KB  22982             80                             2880
max              577.8 KB  22982             80                             2880
stddev           0  B      0                 0                              0
total            577.8 KB  22982             80                             2880

,averagevalue,year contour,elevation_contour,culture_name,month,day,vegetation,field_id,culture_name_encoded,indextype_NDVI,indextype_NDWI,indextype_SAVI,indextype_VARI,district_name_Alamudun district,district_name_Chui district,district_name_Kemin district,district_name_Moskva district,district_name_Panfilov district,district_name_Sokuluk district,district_name_Yssyk-Ata district,district_name_Zhayil district,soil_name_Аллювиальные луговые (пойменные),soil_name_Аллювиальные луговые тугайных лесов и кустарников,soil_name_Горно-долинные каштановые и темно-каштановые,"soil_name_Горно-долинные светло-каштановые, местами в сочетании с лугово-светло-каштановыми слабозасоленными",soil_name_Горные светло-каштановые,soil_name_Комплекс лугово-сероземных и сероземно-луговых солончаковатых (25-30%) и солонцеватых (25-30%),"soil_name_Комплекс сероземно-луговых и луговых (сазовых) солончаковатых (15-30%) и солонцеватых (15-30%), местами вторично засоленных",soil_name_Лугово-сероземные в сочетании с сероземами северными (малокарбонатными) обыкновенными,soil_name_Луговые и лугово-болотные,"soil_name_Сероземно-луговые, местами солончаковатые",soil_name_Сероземы северные (малокарбонатные) обыкновенные,soil_name_Сероземы северные (малокарбонатные) светлые вторично засоленные и заболоченные,"soil_name_Сероземы северные (малокарбонатные) светлые, местами слабо- и среднесолончаковатые",type_culture_name_Озимовая,type_culture_name_Яровая
type,real,int,int,enum,int,int,enum,int,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
mins,-0.715,2020.0,576.0,,2.0,1.0,,1.0,0.0,,,,,,,,,,,,,,,,,,,,,,,,,,,
mean,0.05922648159429118,2020.9525715777565,768.9400400313289,,6.209859890348969,7.058915673135497,,477.60442955356365,11.288660690975544,,,,,,,,,,,,,,,,,,,,,,,,,,,
maxs,1.251,2022.0,1571.0,,10.0,23.0,,996.0,21.0,,,,,,,,,,,,,,,,,,,,,,,,,,,
sigma,0.229099937658748,0.81664675846038,189.23267854808918,,1.9336818804001872,5.476598824475133,,280.6047790559296,6.045564141137502,,,,,,,,,,,,,,,,,,,,,,,,,,,
zeros,40,0,0,,0,0,,0,156,,,,,,,,,,,,,,,,,,,,,,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.217,2020.0,1544.0,Картофель,6.0,4.0,True,986.0,2.0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
1,0.215,2020.0,1544.0,Картофель,8.0,3.0,True,986.0,2.0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
2,0.15,2020.0,1544.0,Картофель,9.0,2.0,True,986.0,2.0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True


In [13]:
train, test = h2odf.split_frame(ratios=[0.8])

In [14]:
train

averagevalue,year contour,elevation_contour,culture_name,month,day,vegetation,field_id,culture_name_encoded,indextype_NDVI,indextype_NDWI,indextype_SAVI,indextype_VARI,district_name_Alamudun district,district_name_Chui district,district_name_Kemin district,district_name_Moskva district,district_name_Panfilov district,district_name_Sokuluk district,district_name_Yssyk-Ata district,district_name_Zhayil district,soil_name_Аллювиальные луговые (пойменные),soil_name_Аллювиальные луговые тугайных лесов и кустарников,soil_name_Горно-долинные каштановые и темно-каштановые,"soil_name_Горно-долинные светло-каштановые, местами в сочетании с лугово-светло-каштановыми слабозасоленными",soil_name_Горные светло-каштановые,soil_name_Комплекс лугово-сероземных и сероземно-луговых солончаковатых (25-30%) и солонцеватых (25-30%),"soil_name_Комплекс сероземно-луговых и луговых (сазовых) солончаковатых (15-30%) и солонцеватых (15-30%), местами вторично засоленных",soil_name_Лугово-сероземные в сочетании с сероземами северными (малокарбонатными) обыкновенными,soil_name_Луговые и лугово-болотные,"soil_name_Сероземно-луговые, местами солончаковатые",soil_name_Сероземы северные (малокарбонатные) обыкновенные,soil_name_Сероземы северные (малокарбонатные) светлые вторично засоленные и заболоченные,"soil_name_Сероземы северные (малокарбонатные) светлые, местами слабо- и среднесолончаковатые",type_culture_name_Озимовая,type_culture_name_Яровая
0.217,2020,1544,Картофель,6,4,True,986,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.15,2020,1544,Картофель,9,2,True,986,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.056,2021,1544,Картофель,5,5,False,987,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.153,2021,1544,Картофель,6,14,True,987,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.111,2021,1544,Картофель,7,9,False,987,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.033,2022,1544,Картофель,4,5,False,988,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.086,2022,1544,Картофель,5,10,False,988,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.192,2022,1544,Картофель,6,14,True,988,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
0.084,2022,1544,Картофель,9,2,False,988,2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
-0.2,2020,1544,Картофель,6,4,False,986,2,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True


In [15]:
aml = H2OAutoML(max_models = 10,
                balance_classes=True)

In [16]:
aml.train(training_frame = train, y = 'culture_name_encoded')

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_3_AutoML_1_20231012_132035


Model Summary: 
    number_of_trees
--  -----------------
    167

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 1.1620532468229808e-09
RMSE: 3.408890210644779e-05
MAE: 1.9469482242975068e-05
RMSLE: 8.636146173667e-06
Mean Residual Deviance: 1.1620532468229808e-09

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 1.4942783539963046e-09
RMSE: 3.865589675581598e-05
MAE: 2.154077186957353e-05
RMSLE: 1.0963710926122796e-05
Mean Residual Deviance: 1.4942783539963046e-09

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     2.15409e-05  1.91585e-06  2.18301e-05   1.88069e-05   2.20863e-05   2.0906e-05    2.40752e-05
mean_residual_deviance  1.49428e-09  1.77643e-10  1.38293e-09   1.26905e-09   1.72239e-09   1.49764e-09   1.59941e-09
mse                     1.49428e-09  1.77643e-10  1.38293e-09   1.26905e-09   1.72239e-09   1.49764e-09   1.59941e-09
r2                      1            0            1             1             1             1             1
residual_deviance       1.49428e-09  1.77643e-10  1.38293e-09   1.26905e-09   1.72239e-09   1.49764e-09   1.59941e-09
rmse                    3.8601e-05   2.30337e-06  3.71877e-05   3.56237e-05   4.15017e-05   3.86993e-05   3.99927e-05
rmsle                   1.09456e-05  7.05791e-07  1.04949e-05   1.0229e-05    1.1571e-05    1.0609e-05    1.1824e-05

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse          training_mae            training_deviance
---  -------------------  ----------  -----------------  ---------------------  ----------------------  ----------------------
     2023-10-12 13:22:56  1.567 sec   0.0                12.357630770389944     10.782052671507532      152.71103825728838
     2023-10-12 13:22:56  1.596 sec   5.0                2.4501908985407557     2.0223696875368073      6.003435439291956
     2023-10-12 13:22:56  1.602 sec   10.0               0.8588916427174578     0.5383064119867763      0.7376948539298931
     2023-10-12 13:22:56  1.608 sec   15.0               0.47165372455270366    0.2660913632390713      0.22245723588443764
     2023-10-12 13:22:56  1.614 sec   20.0               0.2884823636312462     0.16380995143153654     0.08322207412627054
     2023-10-12 13:22:56  1.621 sec   25.0               0.1746332317111034     0.09956114572758817     0.03049676561786393
     2023-10-12 13:22:56  1.628 sec   30.0               0.11179600593573857    0.06804995730856819     0.012498346943183695
     2023-10-12 13:22:56  1.634 sec   35.0               0.0722183589853031     0.04188807930268845     0.005215491374530109
     2023-10-12 13:22:56  1.640 sec   40.0               0.046620945929944306   0.025246597378331712    0.0021735125994027903
     2023-10-12 13:22:56  1.647 sec   45.0               0.02997401713266284    0.017555493080203863    0.0008984417030691655
---  ---                  ---         ---                ---                    ---                     ---
     2023-10-12 13:22:56  1.789 sec   125.0              3.470312219343687e-05  1.9596762369781304e-05  1.2043066899726107e-09
     2023-10-12 13:22:56  1.800 sec   130.0              3.463217340864912e-05  1.9579838829870426e-05  1.199387435006743e-09
     2023-10-12 13:22:56  1.811 sec   135.0              3.458887460943268e-05  1.956966319299822e-05   1.1963902467470567e-09
     2023-10-12 13:22:56  1.822 sec   140.0              3.411535783123745e-05  1.9457765962491286e-05  1.163857639953374e-09
     2023-10-12 13:22:56  1.832 sec   145.0              3.41115087124464e-05   1.946110451123691e-05   1.1635950266393068e-09
     2023-10-12 13:22:56  1.841 se